In [2]:
import cv2
import torch
import numpy as np
from torchvision import transforms
from torchvision.utils import save_image
import xml.etree.ElementTree as ET
from PIL import Image

def VOC2bbox(xml_file: str):

    tree = ET.parse(xml_file)
    root = tree.getroot()

    list_with_all_boxes = []

    for boxes in root.iter('object'):

        filename = root.find('filename').text

        ymin, xmin, ymax, xmax = None, None, None, None

        ymin = int(float(boxes.find("bndbox/ymin").text))
        xmin = int(float(boxes.find("bndbox/xmin").text))
        ymax = int(float(boxes.find("bndbox/ymax").text))
        xmax = int(float(boxes.find("bndbox/xmax").text))

        if(xmax-xmin) == 0 or (ymax-ymin) == 0:
            continue
        else:
            list_with_single_boxes = [[xmin, ymin], [xmax, ymin], [xmax, ymax], [xmin, ymax]]
            list_with_all_boxes.append(list_with_single_boxes)

    return filename, list_with_all_boxes

input_size = 256

idx = "0040"
root_dataset = "../../destijl_dataset/"
design_path = root_dataset + '00_preview/'+idx+".png"
image_xml = root_dataset + "xmls/" + '02_image/'+idx+".xml"
text_xml = root_dataset + "xmls/" + '04_text/'+idx+".xml"
xml_paths = [image_xml, text_xml]

transform = transforms.Compose([
    transforms.Resize((input_size, input_size)),
    transforms.ToTensor(), 
    #transforms.Normalize((0.5,), (0.5,))
    ])

In [5]:
image = cv2.imread(design_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

count = 0
for i in range(2):
    path = xml_paths[i]
    bboxes = VOC2bbox(path)[1]
    for bbox in bboxes:
        x, y = int(bbox[0][0]), int(bbox[0][1])
        z, t = int(bbox[2][0]), int(bbox[2][1])
        cropped_image = Image.fromarray(image[y:t, x:z])
        tensor = transform(cropped_image)
        save_image(tensor, "image_"+ str(count)+".png")
        count += 1


torch.Size([3, 256, 256])
torch.Size([3, 256, 256])
torch.Size([3, 256, 256])
torch.Size([3, 256, 256])
torch.Size([3, 256, 256])
torch.Size([3, 256, 256])
torch.Size([3, 256, 256])
